In [186]:
import pandas as pd
import re
import itertools


def normalize(colname):
    return re.sub(r'[^a-z0-9]', '', colname.lower())

# Trova indici colonne codice (può essere 'codice' o 'mccode')
def find_codice_col(cols):
    normalized_cols = [normalize(c) for c in cols]
    for target in ['codice', 'mccode']:
        if target in normalized_cols:
            return cols[normalized_cols.index(target)]
    raise Exception("Colonna 'codice' o 'mccode' non trovata.")

def parse_quantita(val):
    if pd.isna(val) or str(val).strip() == '':
        return 0
    s = str(val).lower().strip()
    if 'set' in s:
        return 0
    if '+' in s:
        # Somma tutti i numeri separati da +
        nums = [int(part) for part in s.split('+') if part.strip().isdigit()]
        return sum(nums)
    if '(' in s:
        num = s.split('(')[0]
        return int(num) if num.strip().isdigit() else 0
    try:
        return int(float(s))
    except:
        return 0

def expand_model(s):
    results = []

    def recurse(current, rest):
        if not rest:
            results.append(current)
            return

        # Se trova una parentesi, gestisce blocco e ricorsione
        if rest[0] == '(':
            # Trova la parentesi chiusa corrispondente
            close = rest.find(')')
            if close == -1:
                # parentesi non chiusa, tratta come testo normale
                recurse(current + '(', rest[1:])
                return
            inside = rest[1:close]
            after = rest[close+1:]
            # Se dentro c'è uno slash, splitta sulle opzioni
            if '/' in inside:
                options = inside.split('/')
                # Determina la lunghezza da sostituire (come la prima opzione)
                n = len(options[0])
                pre = current[:-n] if n > 0 else current
                recurse(pre,inside + after)
                # Inoltre: anche la variante senza parentesi
                recurse(current, after)
            else:
                # Senza slash: una versione con la sostituzione, una senza
                n = len(inside)
                pre = current[:-n] if n > 0 else current
                recurse(pre + inside, after)  # con sostituzione
                recurse(current, after)       # senza sostituzione
            return

        # Se trova uno slash (fuori da parentesi), split immediato in due rami
        if rest[0] == '/':
            # Variante senza il simbolo prima di '/', variante senza simbolo dopo
            if current:
                recurse(current, rest[2:])            # salta lo slash, prosegue normale
                recurse(current[:-1] + rest[1], rest[2:])  # sostituisce simbolo prima di / con quello dopo
            else:
                recurse('', rest[1:])
            return

        # Altrimenti prosegue normalmente
        recurse(current + rest[0], rest[1:])

    recurse('', s)
    # Unici e ordinati
    seen = set()
    out = []
    for r in results:
        if r and r not in seen:
            out.append(r)
            seen.add(r)
    return out

spareorder = pd.read_excel('output_spareorder.xlsx')  # o carica da dove hai i nomi modello
modelli = spareorder['modello'].astype(str)


MIDEA

In [187]:
# --- Caricamento dei dati ---
df = pd.read_excel(r'..\MIDEA\anagrafica spare parts MWO MIDEA_2018-22.xlsx', header=1)  # header=1 per usare la riga 2 come intestazione

# --- Trova colonne chiave ---
cols = list(df.columns)
col_codice = find_codice_col(cols)
col_stat = [c for c in cols if normalize(c) == 'stat'][0]

modelli_norm = [normalize(m) for m in modelli]
normalized_cols = [normalize(c) for c in cols]
col_modelli = [col for col, norm in zip(cols, normalized_cols) if norm in modelli_norm]

final_rows = []
for idx, row in df.iterrows():
    codice = row[col_codice]
    stat = row[col_stat]
    for col in col_modelli:
        val = row[col]
        if pd.notna(val) and isinstance(val, (int, float)) and val > 0:
            modello = col  # oppure: df.columns[df.columns.get_loc(col)] se vuoi la label esatta della colonna
            final_rows.append({
                'codice': codice,
                'stat': stat,
                'modello': modello,
                'quantità': int(round(val))
            })

df_finale = pd.DataFrame(final_rows)

In [188]:
df_finale

,codice,stat,modello,quantità
0,MWWAC0101,AC01,SM720CWW,1
1,MWWAC0101,AC01,MWHN-20WAG,1
2,MWWAC0101,AC01,SM207S,1
3,MWWAC0101,AC01,MWHN-20WA,1
4,MWWAC0101,AC01,MG2070G,1
...,...,...,...,...
1181,MWWAC0411,AC04,MWHN20W,1
1182,MWWAC0412,AC04,MWHN20B,1
1183,MWWAC0413,AC04,MWHN20BG,1
1184,MWWAC0414,AC04,FM25GKD,1


HISENSE

In [189]:
# --- Caricamento dei dati ---
df = pd.read_excel(r'..\HISENSE\anagrafica HISENSE LVB (All brand) 2022 - 24.xlsx', header=1)  # header=1 per usare la riga 2 come intestazione

# --- Trova colonne chiave ---
cols = list(df.columns)
col_codice = find_codice_col(cols)
col_stat = [c for c in cols if normalize(c) == 'stat'][0]

modelli_norm = [normalize(m) for m in modelli]
normalized_cols = [normalize(c) for c in cols]
col_modelli = [col for col, norm in zip(cols, normalized_cols) if norm in modelli_norm]

final_rows = []
for idx, row in df.iterrows():
    codice = row[col_codice]
    stat = row[col_stat]
    for col in col_modelli:
        val = row[col]
        if pd.notna(val) and isinstance(val, (int, float)) and val > 0:
            modello = col  # oppure: df.columns[df.columns.get_loc(col)] se vuoi la label esatta della colonna
            final_rows.append({
                'codice': codice,
                'stat': stat,
                'modello': modello,
                'quantità': int(round(val))
            })

df_finale = pd.DataFrame(final_rows)

In [190]:
df_finale

,codice,stat,modello,quantità
0,H2128148,L0,SWMH127S,1
1,H2128148,L0,SWMH127S1,1
2,H2128148,L0,SWMH106S,1
3,H2128148,L0,DHLB6010SL,1
4,H2128148,L0,DHLB7012SL,1
...,...,...,...,...
2710,H2125280,NaN,LBHN-6HIS10S,1
2711,H2125280,NaN,WMHN712S,1
2712,H2125280,NaN,LBHN-7HIS12S,1
2713,H2125280,NaN,WMHNMB6010S,1


HOMA

In [191]:
# --- Caricamento dei dati ---
df = pd.read_excel(r'..\HOMA\anagrafica frigo HOMA 2024.1.xlsx')  # header=1 per usare la riga 2 come intestazione


# Trova la colonna codice
col_codice = find_codice_col(df.columns)

# Trova la colonna STAT (usando lo stesso criterio di normalize)
col_stat = [c for c in df.columns if normalize(c) == 'stat'][0]

# Trova la colonna quantità (può essere 'quantità', 'quantity', ecc)
col_quantita = [c for c in df.columns if normalize(c) in ['quantita', 'quantity']][0]

# Trova tutte le colonne modello (nome che inizia per "model ")
col_modelli = [c for c in df.columns if normalize(c).startswith('model')]

final_rows = []
for idx, row in df.iterrows():
    codice = row[col_codice]
    stat = row[col_stat]
    quantita = row[col_quantita]
    for col in col_modelli:
        val = row[col]
        if pd.notna(val) and str(val).strip() != '':
            modello = str(val).strip()
            final_rows.append({
                'codice': codice,
                'stat': stat,
                'modello': modello,
                'quantità': parse_quantita(quantita)
            })

df_finale = pd.DataFrame(final_rows)

# Mantieni solo le righe dove almeno uno dei due (modello o quantità) NON sia zero
df_finale = df_finale[(df_finale['modello'] != '0') & (df_finale['quantità'] != 0)]

In [192]:
expanded_rows = []
for idx, row in df_finale.iterrows():
    modelli_espansi = expand_model(row['modello'])
    for m in modelli_espansi:
        new_row = row.copy()
        new_row['modello'] = m
        expanded_rows.append(new_row)

df_finale_expanded = pd.DataFrame(expanded_rows)


In [193]:
df_finale_expanded

,codice,stat,modello,quantità
2,VAM10010,var,SHMT-46B,1
5,VAM10010,var,DFT-9B,1
6,VAM10010,var,FBNH-420,1
8,VAM10010,var,OFK-10014A1,1
10,VAM10010,var,FMTV-46H,1
...,...,...,...,...
110540,VAM94035,var,ALL TECH,1
110556,VAM94036,var,ALL CHEST FREEZERS TECH,1
110583,VAM94037,var,ALL CHEST AAAMAZE,1
110599,VAM94038,var,ALL AAAMAZE,1


ANAGRAFICA

In [8]:
import pandas as pd
import re

def normalize(colname):
    """Normalizza i nomi delle colonne per una ricerca più semplice."""
    return re.sub(r'[^a-z0-9]', '', colname.lower())

def find_codice_col(cols):
    """Trova la colonna del codice del componente."""
    normalized_cols = [normalize(c) for c in cols]
    for target in ['codice', 'mccode']:
        if target in normalized_cols:
            return cols[normalized_cols.index(target)]
    raise Exception("Colonna 'codice' o 'mccode' non trovata.")

def find_descrizione_col(cols):
    """Trova la colonna della descrizione in italiano."""
    descr_keywords = ['descrizione', 'desrita', 'descrizionericambio','descr','desr']
    normalized_cols = [normalize(c) for c in cols]
    for keyword in descr_keywords:
        for col in normalized_cols:
            if keyword in col:
                return cols[normalized_cols.index(col)]
    return None

def find_english_description_col(cols, col_to_exclude):
    """
    Trova la colonna della descrizione in inglese, escludendo quella già trovata.
    Cerca 'Part name', 'Description', o 'PART DESCRIPTION'.
    """
    eng_descr_keywords = ['partname', 'description', 'partdescription']
    search_cols = [c for c in cols if c != col_to_exclude]
    normalized_cols = [normalize(c) for c in search_cols]
    for keyword in eng_descr_keywords:
        for i, col_norm in enumerate(normalized_cols):
            if keyword in col_norm:
                return search_cols[i]
    return None

def find_price_col(cols):
    """
    Trova la colonna del prezzo. Cerca 'price' o 'prezzo'.
    """
    price_keywords = ['price', 'prezzo']
    normalized_cols = [normalize(c) for c in cols]
    for keyword in price_keywords:
        for i, col_norm in enumerate(normalized_cols):
            if keyword in col_norm:
                return cols[i] # Ritorna il nome della colonna originale
    return None

def parse_quantita(val):
    """Interpreta e pulisce i valori nella colonna quantità."""
    if pd.isna(val) or str(val).strip() == '':
        return 0
    s = str(val).lower().strip()
    if 'set' in s:
        return 0
    if '+' in s:
        nums = [int(part) for part in s.split('+') if part.strip().isdigit()]
        return sum(nums)
    if '(' in s:
        num = s.split('(')[0]
        return int(num) if num.strip().isdigit() else 0
    try:
        return int(float(s))
    except:
        return 0

def expand_model(s):
    """Espande i nomi dei modelli compressi (es. 'A(B/C)' -> 'AB', 'AC', 'A')."""
    results = []
    def recurse(current, rest):
        if not rest:
            results.append(current)
            return
        if rest[0] == '(':
            close = rest.find(')')
            if close == -1:
                recurse(current + '(', rest[1:])
                return
            inside = rest[1:close]
            after = rest[close+1:]
            if '/' in inside:
                options = inside.split('/')
                n = len(options[0])
                pre = current[:-n] if n > 0 else current
                for opt in options:
                    recurse(pre + opt, after)
                recurse(current, after)
            else:
                n = len(inside)
                pre = current[:-n] if n > 0 else current
                recurse(pre + inside, after)
                recurse(current, after)
            return
        if rest[0] == '/':
            if current:
                recurse(current, rest[2:])
                recurse(current[:-1] + rest[1], rest[2:])
            else:
                recurse('', rest[1:])
            return
        recurse(current + rest[0], rest[1:])
    recurse('', s)
    seen = set()
    out = []
    for r in results:
        if r and r not in seen:
            out.append(r)
            seen.add(r)
    return out

# --- Carica i modelli di riferimento ---
spareorder = pd.read_excel('output_spareorder.xlsx')
modelli = spareorder['modello'].astype(str)
modelli_norm = [normalize(m) for m in modelli]

# Funzione per processare file in formato "tabella modelli sulle colonne"
def process_df_tabella(df):
    cols = list(df.columns)
    col_codice = find_codice_col(cols)
    col_stat = [c for c in cols if normalize(c) == 'stat'][0]
    col_descr = find_descrizione_col(cols)
    col_descr_en = find_english_description_col(cols, col_descr)
    col_price = find_price_col(cols) # NUOVO
    
    normalized_cols = [normalize(c) for c in cols]
    col_modelli = [col for col, norm in zip(cols, normalized_cols) if norm in modelli_norm]
    rows = []
    for idx, row in df.iterrows():
        codice = row[col_codice]
        stat = row[col_stat]
        descrizione = row[col_descr] if col_descr and pd.notna(row[col_descr]) else ''
        descrizione_en = row[col_descr_en] if col_descr_en and pd.notna(row[col_descr_en]) else ''
        price = row[col_price] if col_price and pd.notna(row[col_price]) else 0 # NUOVO
        
        for col in col_modelli:
            val = row[col]
            if pd.notna(val) and isinstance(val, (int, float)) and val > 0:
                modello = col
                rows.append({
                    'codice': codice,
                    'stat': stat,
                    'modello': modello,
                    'quantità': int(round(val)),
                    'descrizione': descrizione,
                    'descrizione_en': descrizione_en,
                    'price': price # NUOVO
                })
    return rows

# Funzione per processare file in formato "tabella modelli nelle celle"
def process_df_celle(df):
    cols = list(df.columns)
    col_codice = find_codice_col(cols)
    col_stat = [c for c in cols if normalize(c) == 'stat'][0]
    col_quantita = [c for c in cols if normalize(c) in ['quantita', 'quantity']][0]
    col_modelli = [c for c in cols if normalize(c).startswith('model')]
    col_descr = find_descrizione_col(cols)
    col_descr_en = find_english_description_col(cols, col_descr)
    col_price = find_price_col(cols) # NUOVO
    
    rows = []
    for idx, row in df.iterrows():
        codice = row[col_codice]
        stat = row[col_stat]
        quantita = row[col_quantita]
        descrizione = row[col_descr] if col_descr and pd.notna(row[col_descr]) else ''
        descrizione_en = row[col_descr_en] if col_descr_en and pd.notna(row[col_descr_en]) else ''
        price = row[col_price] if col_price and pd.notna(row[col_price]) else 0 # NUOVO

        for col in col_modelli:
            val = row[col]
            if pd.notna(val) and str(val).strip() != '':
                for modello in expand_model(str(val).strip()):
                    rows.append({
                        'codice': codice,
                        'stat': stat,
                        'modello': modello,
                        'quantità': parse_quantita(quantita),
                        'descrizione': descrizione,
                        'descrizione_en': descrizione_en,
                        'price': price # NUOVO
                    })
    return rows

# --- Elenco file e tipo ---
elenco = [
    {'file': r'..\MIDEA\anagrafica spare parts MWO MIDEA_2018-22.xlsx', 'tipo': 'tabella'},
    {'file': r'..\HISENSE\anagrafica HISENSE LVB (All brand) 2022 - 24.xlsx', 'tipo': 'tabella'},
    {'file': r'..\HOMA\anagrafica frigo HOMA 2024.1.xlsx', 'tipo': 'celle'}
]

final_rows = []

for entry in elenco:
    df = pd.read_excel(entry['file'], header=1 if entry['tipo'] == 'tabella' else 0)
    if entry['tipo'] == 'tabella':
        final_rows += process_df_tabella(df)
    else:
        final_rows += process_df_celle(df)

df_finale = pd.DataFrame(final_rows)

# Filtra: elimina righe con modello '0' o quantità 0
df_finale = df_finale[(df_finale['modello'] != '0') & (df_finale['quantità'] != 0)]

# Ora df_finale contiene tutti i dati, incluse le nuove colonne
print(df_finale.head())
print(f"Colonne finali: {df_finale.columns}")

      codice  stat     modello  quantità        descrizione    descrizione_en  \
0  MWWAC0101  AC01    SM720CWW         1  PIATTO 20L bigger  GLASS TRAY (20L)   
1  MWWAC0101  AC01  MWHN-20WAG         1  PIATTO 20L bigger  GLASS TRAY (20L)   
2  MWWAC0101  AC01      SM207S         1  PIATTO 20L bigger  GLASS TRAY (20L)   
3  MWWAC0101  AC01   MWHN-20WA         1  PIATTO 20L bigger  GLASS TRAY (20L)   
4  MWWAC0101  AC01     MG2070G         1  PIATTO 20L bigger  GLASS TRAY (20L)   

  price  
0  2.13  
1  2.13  
2  2.13  
3  2.13  
4  2.13  
Colonne finali: Index(['codice', 'stat', 'modello', 'quantità', 'descrizione',
       'descrizione_en', 'price'],
      dtype='object')


In [9]:
df_finale

,codice,stat,modello,quantità,descrizione,descrizione_en,price
0,MWWAC0101,AC01,SM720CWW,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
1,MWWAC0101,AC01,MWHN-20WAG,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
2,MWWAC0101,AC01,SM207S,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
3,MWWAC0101,AC01,MWHN-20WA,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
4,MWWAC0101,AC01,MG2070G,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
...,...,...,...,...,...,...,...
126816,VAM94035,var,ALL TECH,1,LOGO TECHLIFE,Trademark TECHLIFE,1.8
126832,VAM94036,var,ALL CHEST FREEZERS TECH,1,LOGO TECHLIFE per chest freezer,Trademark TECHLIFE,1.8
126859,VAM94037,var,ALL CHEST AAAMAZE,1,LOGO AAAMAZE per chest freezer,Trademark AAAMAZE,1.8
126875,VAM94038,var,ALL AAAMAZE,1,LOGO AAAMAZE,Trademark AAAMAZE,1.8


In [10]:
# Tutti i modelli normalizzati nel DataFrame (dopo espansione)
modelli_in_df = set(df_finale['modello'].apply(normalize))

# Tutti i modelli normalizzati di riferimento
modelli_norm_set = set(normalize(m) for m in modelli)

# Trova i modelli che sono nel DataFrame ma non tra quelli di riferimento
modelli_non_trovati = modelli_in_df - modelli_norm_set

# Ora stampa i valori originali (non normalizzati), unici, associati a questi norm
modelli_originali_non_trovati = df_finale[
    df_finale['modello'].apply(lambda x: normalize(x) in modelli_non_trovati)
]['modello'].unique()

print("Modelli presenti nel DataFrame ma NON in modelli di riferimento:")
for m in modelli_originali_non_trovati:
    print(m)


Modelli presenti nel DataFrame ma NON in modelli di riferimento:
FMTV-46N
SDLE-09N
CR FT 50 B
CR FT 60 N
TFFT141
DFT-11NSM1WE1
FTHN-11NSM1WE1
FTHN-11NSM1WF1E0
FTHN-11NSM1WF0E0
FT4HN-14SM1WF0
TT4HN-14SM1WE0
OFK144
CR FT 150 X
CR FT 160 B
SDLE-16SM1XF0
DLT-11NSM1WE1
SDLE-11NSM1WFE0
KHDP-27A
KHDP-27M
KHDP-28A
SHDP-28MA
SHDP-280B
KHDP-27AS
KHDP-27MS
KHDP-28AS
WHDP-287MX
KHDP-29X
KHDP-29S
KHDP-29A
KHDP-29M
KHDP-27X
KHDP-27S
KHDP-28X
KHDP-28S
KHDP-287MS
SHDP-28MX
SHDP-280S
SHDP-284X
DDP-29HX
DDP-29HS
DDP-29H9
DPV-29S
GHDP-2SHDP-229B
GHDP-2GHDP-229B
GHDP-29HDP-229B
DDP-28NH2S
DDHE-2820X
CR-DP 2243 BE
CR-DP 2245 XE
SHDP-28NXE0
SHDP-28NWE0
SHDP-2XE0
SHDP-28N1
SHDP-28NSM1
CR-DP 222245 XBE
CR-DP 222245 XE 
CR-DP 22244  XBE
CR-DP 22244  XE 
CR-DP 224245 XBE
CR-DP 224245 XE 
CR-DP 2243   XBE
CR-DP 2243   XE 
CR-DP 242245 XBE
CR-DP 242245 XE 
CR-DP 24244  XBE
CR-DP 24244  XE 
CR-DP 241245 XBE
CR-DP 241245 XE 
CR-DP 241    XBE
CR-DP 241    XE 
WHDP-28NSM1WE2E2
WHDP-28NSM1WE2E1
WHDP-28NSM1WE2E0
WHDP-2

In [11]:
# Normalizza la colonna 'modello' di df_finale
df_finale['modello_norm'] = df_finale['modello'].apply(normalize)

# Normalizza anche i modelli di riferimento
modelli_norm_set = set(normalize(m) for m in modelli)

# Filtro: tieni solo i modelli che corrispondono (versione rapida ed efficiente)
df_finale = df_finale[df_finale['modello_norm'].isin(modelli_norm_set)].copy()

# (facoltativo) rimuovi la colonna temporanea se non ti serve
df_finale = df_finale.drop(columns=['modello_norm'])


In [12]:
df_finale

,codice,stat,modello,quantità,descrizione,descrizione_en,price
0,MWWAC0101,AC01,SM720CWW,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
1,MWWAC0101,AC01,MWHN-20WAG,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
2,MWWAC0101,AC01,SM207S,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
3,MWWAC0101,AC01,MWHN-20WA,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
4,MWWAC0101,AC01,MG2070G,1,PIATTO 20L bigger,GLASS TRAY (20L),2.13
...,...,...,...,...,...,...,...
126490,NaN,BOX,CFE380SH4WE0,1,cartone imballo,Carton,0
126491,NaN,BOX,CFE380SH4WF0,1,cartone imballo,Carton,0
126492,NaN,BOX,CFHN380SH4WF0,1,cartone imballo,Carton,0
126493,NaN,BOX,HZCF380SH4WE0,1,cartone imballo,Carton,0


In [13]:
df_finale.to_excel('output_anagrafica.xlsx', index=False)